<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_04_3_memory_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 4: LangChain: Chat and Memory**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 4 Material

* Part 4.1: LangChain Conversations [[Video]](https://www.youtube.com/watch?v=Effbhxq07Ag) [[Notebook]](t81_559_class_04_1_langchain_chat.ipynb)
* Part 4.2: Conversation Buffer Window Memory [[Video]](https://www.youtube.com/watch?v=14RgiFVGfAA) [[Notebook]](t81_559_class_04_2_memory_buffer.ipynb)
* **Part 4.3: Conversation Token Buffer Memory** [[Video]](https://www.youtube.com/watch?v=QTe5g2c3bSM) [[Notebook]](t81_559_class_04_3_memory_token.ipynb)
* Part 4.4: Conversation Summary Memory [[Video]](https://www.youtube.com/watch?v=asZQ8Ktqmt8) [[Notebook]](t81_559_class_04_4_memory_summary.ipynb)
* Part 4.5: Persisting Langchain Memory [[Video]](https://www.youtube.com/watch?v=sjCyqqOQcPA) [[Notebook]](t81_559_class_04_5_memory_persist.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Att

# 4.3: Conversation Token Buffer Memory

We previously saw that ConversationBufferWindowMemory can remember the last **k** messages exchanged between the AI and human chatter. Basing the memory on the last number of chat lines has one serious limitation. The underlying LLM limits the chat memory amount based on the context window size, which is measured in tokens, not lines.

In this part, we will see how to use the ConversationTokenBufferMemory. It works almost exactly like the ConversationBufferWindowMemory, except that you specify how many tokens it should keep from the conversation, not how many lines. This technique lets you decide how much of the context window you wish to devote to memory. Remember that the context window must also hold the input prompt and output from the LLM; therefore, some planning is required.

The following code shows how to use a ConversationTokenBufferMemory as the memory of the chat chain.

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'
TEMPLATE = """You are a helpful assistant. Format answers with markdown.

Current conversation:
{history}
Human: {input}
AI:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def begin_conversation(summary_llm):
    # Initialize the OpenAI LLM with your API key
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory and conversation
    memory = ConversationTokenBufferMemory(llm=summary_llm,max_token_limit=2048)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def converse(conversation, prompt):
    output = conversation.invoke(prompt)
    return output['response']

We can now carry on a simple conversation with the LLM, using LangChain to track the conversation memory.

In [ ]:
MODEL = 'gpt-4o-mini'

# Initialize the OpenAI LLM with your API key
llm = ChatOpenAI(
  model=MODEL,
  temperature= 0.3,
  n= 1)

conversation = begin_conversation(llm)
output = converse(conversation, "Hello, what is my name?")
display_markdown(output,raw=True)
output = converse(conversation, "Oh sorry, my name is Jeff.")
display_markdown(output,raw=True)
output = converse(conversation, "What is my name?")
display_markdown(output,raw=True)

I'm sorry, but I don't have access to personal information like your name. How can I assist you today?

Nice to meet you, Jeff! How can I assist you today?

I'm sorry, but I don't have access to personal information like your name. How can I assist you today, Jeff?

## Conversing with the LLM in Markdown

Just as before, we can request that the LLM output be in mardown. This allows code and tables to be represented clearly.

In [ ]:
def chat(conversation, prompt):
  print(f"Human: {prompt}")
  output = converse(conversation, prompt)
  display_markdown(output,raw=True)

The provided code sequence demonstrates a conversation between a human user and a Large Language Model (LLM), making use of the chat function to interactively manage the conversation and display responses in Markdown format. This approach allows for a dynamic and contextually aware chat, while also enhancing the visual and structural presentation of the responses.

In [ ]:
conversation = begin_conversation(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "Give me a table of the 5 most populus cities with population and country.")


Human: What is my name?


I'm sorry, but I don't have access to that information.

Human: Okay, then let me introduce myself, my name is Jeff


Nice to meet you, Jeff! How can I assist you today?

Human: What is my name?


Your name is Jeff. How can I assist you today?

Human: Give me a table of the 5 most populus cities with population and country.


| City         | Population | Country |
|--------------|------------|---------|
| Tokyo        | 37,833,000 | Japan   |
| Delhi        | 30,291,000 | India   |
| Shanghai     | 27,058,000 | China   |
| Sao Paulo    | 22,043,000 | Brazil  |
| Mexico City  | 21,782,000 | Mexico  |

## Constraining the Conversation with a System Prompt

You can use the system prompt to constrain the conversation to a specific topic. Here, we provide a simple agent that will only discuss life insurance.

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'


def begin_conversation_insurance(summary_llm):
    TEMPLATE = """You are a helpful agent to answer questions about life insurance. Do not talk
    about anything else with users. . Format answers with markdown.

    Current conversation:
    {history}
    Human: {input}
    AI:"""
    PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

    # Initialize the OpenAI LLM with your API key
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory and conversation
    memory = ConversationTokenBufferMemory(llm=summary_llm,max_token_limit=2048)
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

conversation = begin_conversation_insurance(llm)
chat(conversation, "What is my name?")
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "What is my name?")
chat(conversation, "What is your favorite programming language?")
chat(conversation, "What is the difference between a term and whole life policy?")

Human: What is my name?


I'm here to help answer questions about life insurance. How can I assist you today?

Human: Okay, then let me introduce myself, my name is Jeff


I'm here to help answer questions about life insurance. How can I assist you today?

Human: What is my name?


I'm here to help answer questions about life insurance. How can I assist you today?

Human: What is your favorite programming language?


I'm here to help answer questions about life insurance. How can I assist you today?

Human: What is the difference between a term and whole life policy?


A term life insurance policy provides coverage for a specific period of time, such as 10, 20, or 30 years. It offers a death benefit if the insured passes away during the term of the policy. On the other hand, a whole life insurance policy provides coverage for the entire lifetime of the insured. It also includes a cash value component that grows over time and can be borrowed against or withdrawn.

##Examining the Conversation Memory

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM.

In [ ]:
print(conversation.memory.load_memory_variables({}))
#conversation.memory.chat_memory.max_token_limit

{'history': "Human: What is my name?\nAI: I'm here to help answer questions about life insurance. How can I assist you today?\nHuman: Okay, then let me introduce myself, my name is Jeff\nAI: I'm here to help answer questions about life insurance. How can I assist you today?\nHuman: What is my name?\nAI: I'm here to help answer questions about life insurance. How can I assist you today?\nHuman: What is your favorite programming language?\nAI: I'm here to help answer questions about life insurance. How can I assist you today?\nHuman: What is the difference between a term and whole life policy?\nAI: A term life insurance policy provides coverage for a specific period of time, such as 10, 20, or 30 years. It offers a death benefit if the insured passes away during the term of the policy. On the other hand, a whole life insurance policy provides coverage for the entire lifetime of the insured. It also includes a cash value component that grows over time and can be borrowed against or withdr

In [ ]:
chat(conversation, "What type of policy did I ask you about?")

Human: What type of policy did I ask you about?


You asked about the difference between a term and whole life policy.

## Overloading the Memory

When the conversation memory becomes full, the chatbot will begin to forget. For the ConversationTokenBufferMemory memory type, the oldest history will first be lost; there is no notion of importance.

In [ ]:
conversation = begin_conversation(llm)
chat(conversation, "Okay, then let me introduce myself, my name is Jeff")
chat(conversation, "You have ONE JOB! Remember that my favorite color is blue.")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")
for i in range(70):
  chat(conversation, f"You need to remember fact #{i}")
chat(conversation, "Do you remember my name?")
chat(conversation, "Do you remember my favorite color?")

Human: Okay, then let me introduce myself, my name is Jeff


Nice to meet you, Jeff! How can I assist you today?

Human: You have ONE JOB! Remember that my favorite color is blue.


Got it, Jeff! Your favorite color is blue. How can I assist you today?

Human: Do you remember my name?


...

I will remember that fact #69 is important to you, Jeff. How can I assist you today?

Human: Do you remember my name?


Yes, your name is Jeff. How can I assist you today?

Human: Do you remember my favorite color?


I'm sorry, I don't have that information. How can I assist you today?

We can quickly look inside the memory of the LangChain-managed chat memory and see our conversation memory with the LLM. It becomes evident why it forgot.

In [ ]:
print(conversation.memory.buffer)

Human: You need to remember fact #15
AI: I will remember that fact #15 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #16
AI: I will remember that fact #16 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #17
AI: I will remember that fact #17 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #18
AI: I will remember that fact #18 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #19
AI: I will remember that fact #19 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #20
AI: I will remember that fact #20 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #21
AI: I will remember that fact #21 is important to you, Jeff. How can I assist you today?
Human: You need to remember fact #22
AI: I will remember that fact #22 is important to you, Jeff. How can I assist you